In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil
import tensorflow as tf
import pathlib
import PIL
import time
import zipfile
import random
from tensorflow import keras as ks
from tensorflow.keras.layers import *
from keras.utils.np_utils import to_categorical 
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import warnings
from tqdm.notebook import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re
import time
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
warnings.filterwarnings('ignore')

In [3]:
train_df=pd.read_csv("../input/businessandai-competition/Xy_train.csv", sep=";")
test_df=pd.read_csv("../input/businessandai-competition/X_test.csv", sep=";")
sub=pd.read_csv("../input/businessandai-competition/submission_sample.csv", sep=";")

In [4]:
train_df.head()

,_id,categ_0,txt_0,categ_1,num_0,categ_2,categ_4,categ_5,num_5,categ_8,...,categ_6,categ_7,txt_3,num_4,categ_3,txt_1,num_6,num_2,num_7,y
0,625267cf0a63071b21c47e2b,2,"a:o@`''E]@W@`'']@eBcqK~@mo""c<@1!)@`RRK@Kw",4,168000,1,0,49,NaN,4,...,5,2,<o<,2008.0,2368,<o<,NaN,NaN,NaN,4500.0
1,625666e07a2d1ced83779994,2,"a:o@eKc<:`@Z!)@K@""]:@R:e:c<@Oq'Yeoee@+:`DKoee@...",4,50,3,0,50,115.0,41,...,5,2,"gEO+3'KccKDEb@Eq'""3+oK'3q@]'@R:K'ee'Eb@Eqo]oq@...",2022.0,2368,<o<,NaN,998.0,NaN,22133.0
2,61b9ed1f7e264de96093187b,1,"a:o@eKc<:`@+o3<`D@']:K:c<@Z!)@Kd""]:@Z1)",4,85717,3,0,34,115.0,41,...,1,2,g}`oY'qo@]Eo:]'@o3@eKoK:c<<'Y'<K}b@}`oBK'3qe@]...,2017.0,2296,"R'D:`3+'@`c<Kqc+'@d@""oqo<K:'@;@Yc:e@:<`+3e'@G'...",NaN,NaN,6.0,11490.0
3,624ac4947e9fdee78b7eba63,2,"a:o@q:c@Z!)@K@""]:@Z1)@B+oK:<3Y@']:K:c<",4,30000,3,0,49,NaN,33,...,10,1,<o<,2017.0,2368,<o<,NaN,NaN,NaN,14500.0
4,6254d841bef56d2c1cf934ea,2,"a:o@eBcqKo""'@hoo@e'q:'",2,173000,3,0,49,NaN,39,...,8,1,<o<,2013.0,2368,<o<,NaN,NaN,NaN,10000.0


In [5]:
test_df.head()

,_id,categ_0,txt_0,categ_1,num_0,categ_2,categ_4,categ_5,num_5,categ_8,...,num_10,categ_6,categ_7,txt_3,num_4,categ_3,txt_1,num_6,num_2,num_7
0,6251210db22e614680cef17b,2,a:o@`oq'<e@@@1!)@`RRK@w@Kqo@o:q`c,4,168982,3,0,49,NaN,1,...,20.0,11,2,<o<,2007.0,2368,<o<,NaN,NaN,NaN
1,609a41b746383f48ea7ed9c4,2,"a:o@eBcqKo""'@Z!?@`q]:@ZZu@:e""@,w1@o`K:R'",2,74500,3,0,19,119.0,39,...,999.0,1,1,gEo:q@`c<]:K:c<<'Eb@E`oY'qo@]'@q'`3+Eb@EO+3'Kc...,NaN,2368,<o<,NaN,NaN,NaN
2,609b46c346383f48ea85a360,2,"a:o@q:c@Bq'Y:3Y@Z!,+@Z)_",4,82996,3,0,34,114.0,33,...,999.0,1,1,<o<,NaN,2368,<o<,NaN,NaN,NaN
3,609a42ee46383f48ea7ef087,2,a:o@B:`o<Kc@Z!)@;;`D@o`K:R'@uB@`+:Y@Yo<3@3eO@O...,4,36470,3,0,19,105.0,25,...,999.0,1,1,gEO+3'KccKDEb@Ecq]:<oK'3q@]'@Ocq]Es,2016.0,2368,<o<,NaN,NaN,NaN
4,609a42cd46383f48ea7eed93,2,"a:o@eBcqKo""'@1!)@`q]:@Z,)@,w1@.oB@3qOo<@q:]'q",2,185000,3,0,51,184.0,39,...,999.0,1,1,gEo:q@`c<]:K:c<<'Es,NaN,2368,<o<,NaN,NaN,NaN


In [6]:
test_df.shape

(15000, 27)

In [7]:
Id=test_df["_id"]

In [8]:
train_df.shape

(45000, 28)

In [9]:
train_df["y"].nunique()

5457

In [10]:
train_df.isna().sum().sort_values(ascending =False).head(20)

num_6      43318
num_7      42688
num_9      41739
num_8      35759
num_2      35674
num_5      32416
num_3      30466
num_1      24203
num_4       5276
num_10      1536
y              1
txt_1          0
categ_3        0
txt_3          0
categ_7        0
categ_6        0
_id            0
categ_9        0
date_0         0
categ_0        0
dtype: int64

In [11]:
train_df.isna().sum().sort_values(ascending =False).head(20)/len(train_df)

num_6      0.962622
num_7      0.948622
num_9      0.927533
num_8      0.794644
num_2      0.792756
num_5      0.720356
num_3      0.677022
num_1      0.537844
num_4      0.117244
num_10     0.034133
y          0.000022
txt_1      0.000000
categ_3    0.000000
txt_3      0.000000
categ_7    0.000000
categ_6    0.000000
_id        0.000000
categ_9    0.000000
date_0     0.000000
categ_0    0.000000
dtype: float64

In [12]:
train_df.drop(["_id","num_6","num_7","num_9","num_8","num_2","num_5","num_3","num_1","txt_0","txt_1","txt_2","txt_3"], axis=1, inplace=True)
test_df.drop(["_id","num_6","num_7","num_9","num_8","num_2","num_5","num_3","num_1","txt_0","txt_1","txt_2","txt_3"], axis=1, inplace=True)


In [13]:
train_df.shape

(45000, 15)

In [14]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   categ_0  45000 non-null  int64  
 1   categ_1  45000 non-null  int64  
 2   num_0    45000 non-null  int64  
 3   categ_2  45000 non-null  int64  
 4   categ_4  45000 non-null  int64  
 5   categ_5  45000 non-null  int64  
 6   categ_8  45000 non-null  int64  
 7   date_0   45000 non-null  object 
 8   categ_9  45000 non-null  int64  
 9   num_10   43464 non-null  float64
 10  categ_6  45000 non-null  int64  
 11  categ_7  45000 non-null  int64  
 12  num_4    39724 non-null  float64
 13  categ_3  45000 non-null  int64  
 14  y        44999 non-null  float64
dtypes: float64(3), int64(11), object(1)
memory usage: 5.1+ MB


In [15]:
import datetime as dt
train_df["date_0"]=pd.to_datetime(train_df["date_0"])
train_df['Date_year'] =train_df["date_0"].dt.year
train_df['Date_month'] = train_df["date_0"].dt.month
train_df['Date_day'] = train_df["date_0"].dt.day
train_df.drop("date_0", axis=1, inplace=True)

In [16]:
test_df["date_0"]=pd.to_datetime(test_df["date_0"])
test_df['Date_year'] =test_df["date_0"].dt.year
test_df['Date_month'] = test_df["date_0"].dt.month
test_df['Date_day'] = test_df["date_0"].dt.day
test_df.drop("date_0", axis=1, inplace=True)

In [17]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   categ_0     45000 non-null  int64  
 1   categ_1     45000 non-null  int64  
 2   num_0       45000 non-null  int64  
 3   categ_2     45000 non-null  int64  
 4   categ_4     45000 non-null  int64  
 5   categ_5     45000 non-null  int64  
 6   categ_8     45000 non-null  int64  
 7   categ_9     45000 non-null  int64  
 8   num_10      43464 non-null  float64
 9   categ_6     45000 non-null  int64  
 10  categ_7     45000 non-null  int64  
 11  num_4       39724 non-null  float64
 12  categ_3     45000 non-null  int64  
 13  y           44999 non-null  float64
 14  Date_year   45000 non-null  int64  
 15  Date_month  45000 non-null  int64  
 16  Date_day    45000 non-null  int64  
dtypes: float64(3), int64(14)
memory usage: 5.8 MB


In [18]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   categ_0     15000 non-null  int64  
 1   categ_1     15000 non-null  int64  
 2   num_0       15000 non-null  int64  
 3   categ_2     15000 non-null  int64  
 4   categ_4     15000 non-null  int64  
 5   categ_5     15000 non-null  int64  
 6   categ_8     15000 non-null  int64  
 7   categ_9     15000 non-null  int64  
 8   num_10      14536 non-null  float64
 9   categ_6     15000 non-null  int64  
 10  categ_7     15000 non-null  int64  
 11  num_4       13237 non-null  float64
 12  categ_3     15000 non-null  int64  
 13  Date_year   15000 non-null  int64  
 14  Date_month  15000 non-null  int64  
 15  Date_day    15000 non-null  int64  
dtypes: float64(2), int64(14)
memory usage: 1.8 MB


In [19]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
estimator=RandomForestRegressor(max_depth=8)
mice = IterativeImputer(estimator=estimator,random_state=11,skip_complete=True)
imp_train_df=mice.fit_transform(train_df)
imp_train_df=pd.DataFrame(imp_train_df,columns=train_df.columns)

In [20]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
imp_test_df=mice.fit_transform(test_df)
imp_test_df=pd.DataFrame(imp_test_df,columns=test_df.columns)

In [21]:
imp_train_df.isna().sum()

categ_0       0
categ_1       0
num_0         0
categ_2       0
categ_4       0
categ_5       0
categ_8       0
categ_9       0
num_10        0
categ_6       0
categ_7       0
num_4         0
categ_3       0
y             0
Date_year     0
Date_month    0
Date_day      0
dtype: int64

In [22]:
imp_train_df["mean"]=imp_train_df.mean(axis=1)
imp_train_df["count"]=imp_train_df.count(axis=1)
imp_train_df["std"]=imp_train_df.std(axis=1)
imp_train_df["min"]=imp_train_df.min(axis=1)
imp_train_df["max"]=imp_train_df.max(axis=1)
imp_train_df["kurt"]=imp_train_df.kurt(axis=1)
imp_train_df["skew"]=imp_train_df.skew(axis=1)
imp_train_df["q1"]=imp_train_df.quantile(q=0.1,axis=1)
imp_train_df["q2"]=imp_train_df.quantile(q=0.2,axis=1)
imp_train_df["q25"]=imp_train_df.quantile(q=0.25,axis=1)
imp_train_df["q3"]=imp_train_df.quantile(q=0.3,axis=1)
imp_train_df["q4"]=imp_train_df.quantile(q=0.4,axis=1)
imp_train_df["q5"]=imp_train_df.quantile(q=0.5,axis=1)
imp_train_df["q6"]=imp_train_df.quantile(q=0.6,axis=1)
imp_train_df["q7"]=imp_train_df.quantile(q=0.7,axis=1)
imp_train_df["q75"]=imp_train_df.quantile(q=0.75,axis=1)
imp_train_df["q8"]=imp_train_df.quantile(q=0.8,axis=1)
imp_train_df["q9"]=imp_train_df.quantile(q=0.9,axis=1)

In [29]:
imp_test_df["mean"]=imp_test_df.mean(axis=1)
imp_test_df["count"]=imp_test_df.count(axis=1)
imp_test_df["std"]=imp_test_df.std(axis=1)
imp_test_df["min"]=imp_test_df.min(axis=1)
imp_test_df["max"]=imp_test_df.max(axis=1)
imp_test_df["kurt"]=imp_test_df.kurt(axis=1)
imp_test_df["skew"]=imp_test_df.skew(axis=1)
imp_test_df["q1"]=imp_test_df.quantile(q=0.1,axis=1)
imp_test_df["q2"]=imp_test_df.quantile(q=0.2,axis=1)
imp_test_df["q25"]=imp_test_df.quantile(q=0.25,axis=1)
imp_test_df["q3"]=imp_test_df.quantile(q=0.3,axis=1)
imp_test_df["q4"]=imp_test_df.quantile(q=0.4,axis=1)
imp_test_df["q5"]=imp_test_df.quantile(q=0.5,axis=1)
imp_test_df["q6"]=imp_test_df.quantile(q=0.6,axis=1)
imp_test_df["q7"]=imp_test_df.quantile(q=0.7,axis=1)
imp_test_df["q75"]=imp_test_df.quantile(q=0.75,axis=1)
imp_test_df["q8"]=imp_test_df.quantile(q=0.8,axis=1)
imp_test_df["q9"]=imp_test_df.quantile(q=0.9,axis=1)

In [30]:
# import caalltegory_encoders as ce
# ordenc=ce.OrdinalEncoder(cols=['Espèce'])
# train_df=ordenc.fit_transform(train_df)
# Test=ordenc.fit_transform(Test)

In [33]:
y_train=imp_train_df["y"]

In [34]:
imp_train_df=imp_train_df.drop("y",axis=1)

In [26]:
# X_train = np.array(train)
# y_train = np.array(train_df[["target"]])
# X_test = np.array(test_df)

In [35]:
imp_train_df.shape, imp_test_df.shape

((45000, 34), (15000, 34))

In [36]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X_train = mms.fit_transform(imp_train_df)
X_test = mms.transform(imp_test_df)


In [37]:
y_train=y_train.to_list()

In [38]:
y_train=np.array(y_train).reshape(-1,1)

In [39]:
mms.fit_transform(y_train)

array([[0.03570663],
       [0.17565219],
       [0.09118326],
       ...,
       [0.25864491],
       [0.18897769],
       [0.20947785]])

In [40]:
len(y_train)

45000

In [41]:
X_test.shape, X_train.shape

((15000, 34), (45000, 34))

In [42]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X_train, y_train, test_size=0.1)

In [43]:
# from sklearn.decomposition import PCA
# pca = PCA()
# rd_xtrain = pca.fit_transform(xtrain)
# rd_xtest = pca.transform(xtest)

In [44]:
# x_test=pca.fit_transform(X_test)[:,:12]

In [46]:
# explained_variance=pca.explained_variance_ratio_
# explained_variance.shape

In [ ]:
# with plt.style.context('dark_background'):
#     plt.figure(figsize=(12, 6))

#     plt.bar(range(16), explained_variance, alpha=0.5, align='center',label='individual explained variance')
#     plt.ylabel('Explained variance ratio')
#     plt.xlabel('Principal components')
#     plt.legend(loc='best')
#     plt.tight_layout()

In [ ]:
# xtrain_ = rd_xtrain[:, :12]
# xtest_ = rd_xtest[:, :12]

In [ ]:
# xtrain_.shape, xtest_.shape

****

In [47]:
from sklearn.ensemble import RandomForestRegressor
Rf = RandomForestRegressor(min_samples_split=15)
Rf.fit(xtrain, ytrain)
ypred=Rf.predict(xtest)

In [48]:
from sklearn.metrics import mean_squared_error
mean_squared_error(ytest, ypred)

2643601.264135854

In [49]:
mean_squared_error(ytest, ypred, squared=True)

2643601.264135854

In [51]:
Rf.fit(xtest, ytest)
ypred=Rf.predict(xtest)

In [52]:
mean_squared_error(ytest, ypred, squared=True)

2716681.932567956

In [53]:
# y_pred4 = Rf.predict(x_test) 
# y_pred_original4 = mms.inverse_transform(y_pred4.reshape(-1,1))
# y_pred_original4=y_pred_original4.tolist()
# y_pred_original4=[i[0] for i in y_pred_original4]
# submission = pd.DataFrame({'_id': Id,'y_pred': y_pred_original4})
# submission.to_csv('submission4.csv', index=False)

****

In [54]:
from xgboost import XGBRegressor
model = XGBRegressor(objective='reg:squarederror')
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model,xtrain,ytrain, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise')
print('MSE: %.8f (%53f)' % (mean(n_scores), std(n_scores)))

MSE: -325294.90759076 (                                         77164.492912)


In [56]:
model.fit(xtrain, ytrain)
yhat = model.predict(xtest)

In [57]:
mean_squared_error(ytest, yhat, squared=True)

302939.9254095129

In [ ]:
# model.fit(X_train, y_train)
# y_pred4 = model.predict(X_test) 
# y_pred_original4 = mms.inverse_transform(y_pred4.reshape(-1,1))
# y_pred_original4=y_pred_original4.tolist()
# y_pred_original4=[i[0] for i in y_pred_original4]
# submission = pd.DataFrame({'Place_ID X Date	': Id,'y_pred': y_pred_original4})
# submission.to_csv('submission12.csv', index=False)

****

In [58]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
model1 = LGBMRegressor()
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model1, xtrain, ytrain, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise')
print('MAE: %.5f (%.5f)' % (mean(n_scores), std(n_scores)))

MAE: -614774.15307 (362103.76658)


In [59]:
model1 = LGBMRegressor()
model1.fit(xtrain, ytrain)
yhatt = model1.predict(xtest)
mean_squared_error(ytest, yhatt)

343915.580513027

In [ ]:
# model1.fit(X_train, y_train)
# y_pred4 = model1.predict(X_test) 
# y_pred_original4 = mms.inverse_transform(y_pred4.reshape(-1,1))
# y_pred_original4=y_pred_original4.tolist()
# y_pred_original4=[i[0] for i in y_pred_original4]
# submission = pd.DataFrame({'_id	': Id,'y_pred': y_pred_original4})
# submission.to_csv('submission13.csv', index=False)

In [60]:
from sklearn import  ensemble
params = {
    "n_estimators": 500,
    "max_depth": 8,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": "squared_error",
}
reg = ensemble.GradientBoostingRegressor(**params)
reg.fit(xtrain, ytrain)
mse = mean_squared_error(ytest, reg.predict(xtest))
mse

2487049.7598281065

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

In [ ]:
# y_pred4 = reg.predict(X_test) 
# y_pred_original4 = mms.inverse_transform(y_pred4.reshape(-1,1))
# y_pred_original4=y_pred_original4.tolist()
# y_pred_original4=[i[0] for i in y_pred_original4]
# submission = pd.DataFrame({'_id': Id,'y_pred': y_pred_original4})
# submission.to_csv('submission12.csv', index=False)